In [1]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/sample_data/06_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)
train_set, dev_set = train_test_split(train_set, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()
X_valid = dev_set.drop('Premium Amount', axis=1)
y_valid = dev_set['Premium Amount'].copy()

In [4]:
import tensorflow.keras.backend as K

# def root_mean_squared_log_error(y_true, y_pred):
#     msle = tf.keras.losses.MeanSquaredLogarithmicError()
#     return K.sqrt(msle(y_true, y_pred))

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(tf.keras.losses.MeanSquaredError()(y_true, y_pred))

In [5]:
%pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.0 MB/s eta 0:00:00


In [6]:
X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)
X_valid = tf.convert_to_tensor(X_valid)
y_valid = tf.convert_to_tensor(y_valid)

In [8]:
import keras_tuner as kt

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=4, max_value=8, default=4)
    n_neurons = hp.Int("n_neurons", min_value=32, max_value=64)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["adam", 'NAG', 'RMSprop'])

    if optimizer == "NAG":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True, weight_decay=1e-4)
    elif optimizer == "RMSprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, rho=0.9, weight_decay=1e-4)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, weight_decay=1e-4)

    model = tf.keras.Sequential()

    for _ in range(n_hidden):
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1))

    model.compile(loss=root_mean_squared_error, optimizer=optimizer)
    return model

In [9]:
random_search_tuner = kt.RandomSearch(
    build_model, objective='val_loss', max_trials=12, overwrite=True,
    directory='kaggle_competition', project_name='my_rnd_search', seed=42
)
random_search_tuner.search(X_train, y_train, epochs=18, validation_data=(X_valid, y_valid), batch_size=100)

Trial 12 Complete [00h 10m 10s]
val_loss: 1.0794402360916138

Best val_loss So Far: 1.0737615823745728
Total elapsed time: 01h 50m 07s


In [10]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 74 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 66 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [11]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values

{'n_hidden': 8,
 'n_neurons': 43,
 'learning_rate': 0.0012482904754698163,
 'optimizer': 'adam'}

In [12]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 01 summary
Hyperparameters:
n_hidden: 8
n_neurons: 43
learning_rate: 0.0012482904754698163
optimizer: adam
Score: 1.0737615823745728


In [13]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()
X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

In [14]:
# Reshape y_test if necessary
y_test = tf.reshape(y_test, (-1, 1))

# Convert y_test to numpy array
y_test = y_test.numpy()

rmsle_test = best_model.evaluate(X_test, y_test)

7500/7500 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 1.0610


In [15]:
# save final prediction
test_set = pd.read_csv('/content/sample_data/06_test.csv')
del test_set['Premium Amount']
test_set = tf.convert_to_tensor(test_set)
y_pred = best_model.predict(test_set)

25000/25000 ━━━━━━━━━━━━━━━━━━━━ 44s 2ms/step


In [16]:
import numpy as np

original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
y_pred = y_pred.flatten()
y_pred = np.exp(y_pred)
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('keras_sequential_10.csv', index=False)